In [45]:
# Importation des bibliothèques qu’on va utiliser
import requests
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [46]:
#Fonction qui permet d’explorer les données
def creat_unique(df):
        
    df_unique = pd.DataFrame(columns=['Column_name','Data_type', 'Number_of_unique','Number_of_missing', 'Unique_values'])

    # loop through the columns in the other dataframe
    for col in df.columns:
        # get the number of unique values in the column
        num_unique = df[col].nunique()

        # add the unique values as a list to the 'Unique_values' column if num_unique <= 5
        if num_unique <= 15:
            unique_vals = list(df[col].unique())
        else:
            unique_vals = "More than 15 unique vales"

        # get the data type of the column
        data_type = df[col].dtype

        # count the number of missing values in the column
        num_missing = df[col].isnull().sum()

        # append a row to the empty dataframe with the column name, number of unique values, unique values, and data type
        df_unique = pd.concat([df_unique,pd.DataFrame([{'Column_name': col, 'Number_of_unique': num_unique, 'Unique_values': unique_vals, 'Data_type':
                                      data_type, 'Number_of_missing': num_missing}])])
    return df_unique

Le simulateur de DPE en fonction d’informations accessibles pour le propriétaire et le locataire: 

In [47]:
#Importation des colonnes utilisées pour la simulation

size = 10000
api_root = "https://koumoul.com/data-fair/api/v1/datasets/dpe-france/lines"
format_api = f"{api_root}?page=1&after=10&format=json&q_mode=simple&"
url_api = "https://koumoul.com/data-fair/api/v1/datasets/dpe-v2-logements-existants/lines?size=10000&select=Etiquette_DPE%2CType_b%C3%A2timent%2CP%C3%A9riode_construction%2CHauteur_sous-plafond%2CSurface_habitable_logement%2CCode_postal_(BAN)%2CType_%C3%A9nergie_n%C2%B01%2CType_%C3%A9nergie_n%C2%B02%2CType_%C3%A9nergie_n%C2%B03%2CIsolation_toiture_(0%2F1)%2CQualit%C3%A9_isolation_murs%2CQualit%C3%A9_isolation_plancher_bas%2CType_%C3%A9nergie_principale_chauffage%2CType_%C3%A9nergie_principale_ECS%2CType_ventilation"
req = requests.get(url_api)
wb = req.json()
df = pd.json_normalize(wb["results"])
df


,Type_énergie_n°3,Type_énergie_n°1,Type_énergie_n°2,Type_ventilation,Isolation_toiture_(0/1),Qualité_isolation_plancher_bas,Qualité_isolation_murs,Code_postal_(BAN),Hauteur_sous-plafond,Surface_habitable_logement,Période_construction,Type_bâtiment,Type_énergie_principale_ECS,Type_énergie_principale_chauffage,Etiquette_DPE,_score
0,Électricité,Gaz naturel,Bois – Bûches,Ventilation par ouverture des fenêtres,False,très bonne,insuffisante,25000,2.9,220.8,avant 1948,appartement,Gaz naturel,Gaz naturel,D,None
1,NaN,Gaz naturel,Électricité,NaN,NaN,insuffisante,insuffisante,56600,2.5,51.0,1948-1974,appartement,Gaz naturel,Électricité,D,None
2,NaN,Réseau de Chauffage urbain,Électricité,VMC SF Hygro B après 2012,NaN,moyenne,insuffisante,78360,2.5,61.0,1948-1974,appartement,Réseau de Chauffage urbain,Réseau de Chauffage urbain,C,None
3,Électricité,Réseau de Chauffage urbain,Gaz naturel,NaN,NaN,insuffisante,insuffisante,44800,2.5,47.5,1975-1977,appartement,NaN,NaN,C,None
4,NaN,Gaz naturel,Électricité,NaN,NaN,insuffisante,insuffisante,75018,2.5,47.0,avant 1948,appartement,Électricité,Électricité,G,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,NaN,Gaz naturel,Électricité,VMC SF Hygro A de 2001 à 2012,True,moyenne,moyenne,62149,2.6,83.8,avant 1948,maison,Gaz naturel,Gaz naturel,D,None
9996,NaN,Gaz naturel,Gaz naturel,NaN,NaN,insuffisante,insuffisante,62400,2.5,84.2,1948-1974,maison,Gaz naturel,Gaz naturel,E,None
9997,NaN,Gaz naturel,Électricité,VMC Basse pression Hygro A,NaN,insuffisante,insuffisante,92500,2.5,66.0,1948-1974,appartement,Gaz naturel,Gaz naturel,D,None
9998,NaN,Gaz naturel,Électricité,VMC SF Auto réglable de 2001 à 2012,True,moyenne,moyenne,28630,2.5,73.1,2001-2005,maison,Gaz naturel,Gaz naturel,D,None


In [48]:
creat_unique(df)

,Column_name,Data_type,Number_of_unique,Number_of_missing,Unique_values
0,Type_énergie_n°3,object,11,7639,"[Électricité, nan, Bois – Bûches, GPL, Gaz nat..."
0,Type_énergie_n°1,object,11,0,"[Gaz naturel, Réseau de Chauffage urbain, Élec..."
0,Type_énergie_n°2,object,13,1725,"[Bois – Bûches, Électricité, Gaz naturel, nan,..."
0,Type_ventilation,object,36,3116,More than 15 unique vales
0,Isolation_toiture_(0/1),object,2,6450,"[False, nan, True]"
0,Qualité_isolation_plancher_bas,object,4,101,"[très bonne, insuffisante, moyenne, bonne, nan]"
0,Qualité_isolation_murs,object,4,0,"[insuffisante, bonne, très bonne, moyenne]"
0,Code_postal_(BAN),object,2889,73,More than 15 unique vales
0,Hauteur_sous-plafond,float64,40,0,More than 15 unique vales
0,Surface_habitable_logement,float64,1695,156,More than 15 unique vales
